<a href="https://colab.research.google.com/github/arguntolga/ts_jupyter/blob/main/gspread_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code example: Export World Happiness Report 2023 to Google Spreadsheet using Google Colab

https://worldhappiness.report/data/

## Load XLS file with 2023 report data into Pandas Dataframe

In [ ]:
import pandas as pd

In [ ]:
complete_df = pd.read_excel("https://happiness-report.s3.amazonaws.com/2023/DataForFigure2.1WHR2023.xls")

In [ ]:
df = complete_df.sort_values("Ladder score", ascending=False)[[
    "Country name", "Ladder score", "Logged GDP per capita", "Social support",
    "Healthy life expectancy", "Freedom to make life choices",
    "Generosity", "Perceptions of corruption"]]

df

,Country name,Ladder score,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,Finland,7.8042,10.792010,0.968770,71.149994,0.961408,-0.018824,0.181745
1,Denmark,7.5864,10.962164,0.954112,71.250145,0.933533,0.134242,0.195814
2,Iceland,7.5296,10.895531,0.982533,72.050018,0.936349,0.210987,0.667848
3,Israel,7.4729,10.638705,0.943344,72.697205,0.808866,-0.023080,0.708094
4,Netherlands,7.4030,10.942279,0.930499,71.550018,0.886875,0.212686,0.378929
...,...,...,...,...,...,...,...,...
132,Congo (Kinshasa),3.2072,7.006671,0.651610,55.375000,0.663798,0.085998,0.833752
133,Zimbabwe,3.2035,7.640998,0.689918,54.049889,0.654055,-0.046230,0.765582
134,Sierra Leone,3.1376,7.394014,0.555251,54.899853,0.660367,0.104929,0.857780
135,Lebanon,2.3922,9.477677,0.529754,66.148819,0.473900,-0.140915,0.891104


## Install gspread library

gspread is a Python API for Google Sheets.

https://docs.gspread.org/en/latest/

In [ ]:
from google.colab import auth
from google.auth import default

try:
  import gspread
except ModuleNotFoundError:
  if 'google.colab' in str(get_ipython()):
    %pip install gspread
  import gspread

## Authenticate user to provide access to Google API

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open spreadsheet
You need to create new spreadsheet named "gspread-test" in your Google Drive

In [ ]:
worksheet = gc.open("gspread-test")
sheet1 = worksheet.sheet1

copy dataframe data to Google Spreadsheet

In [ ]:
sheet1.update([df.columns.values.tolist()] + df.fillna(-1).values.tolist())

{'spreadsheetId': '15f4zO_W-LZCwkAnmCnpD9vHJOz6aOdQC0fdQ938frOU',
 'updatedRange': 'Sheet1!A1:H138',
 'updatedRows': 138,
 'updatedColumns': 8,
 'updatedCells': 1104}

You can also set cell formulas. For example, add image to cell using `image() `function


In [ ]:
sheet1.update('I1', '=image("https://upload.wikimedia.org/wikipedia/commons/8/85/Smiley.svg")', value_input_option='USER_ENTERED')

{'spreadsheetId': '15f4zO_W-LZCwkAnmCnpD9vHJOz6aOdQC0fdQ938frOU',
 'updatedRange': 'Sheet1!I1',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

# Styling spreadsheet
## Setting column header styling

Helper function to convert numeric column indexes into spread column name. For example column with index 3 should be converted into column "D". Thanks to sundar nataraj for providing [code snippet on Stack Overflow](https://stackoverflow.com/a/23862195/171847
)

In [ ]:
def index_to_spreadsheet_column(column_int, start_index=0):
  # https://stackoverflow.com/a/23862195/171847
  letter = ''
  while column_int > 25 + start_index:
      letter += chr(65 + int((column_int-start_index)/26) - 1)
      column_int = column_int - (int((column_int-start_index)/26))*26
  letter += chr(65 - start_index + (int(column_int)))
  return letter

print(index_to_spreadsheet_column(3))

D


find last column in spreadsheet

In [ ]:
last_spreadsheet_column = index_to_spreadsheet_column(len(df.columns)-1)
last_spreadsheet_column

'H'

format column headers with gray background end text in bold

In [ ]:
sheet1.format(f"A1:{last_spreadsheet_column}1", {
    "backgroundColor": {
      "red": 100.0,
      "green": 100.0,
      "blue": 100.0
    },
    "horizontalAlignment": "CENTER",
    "textFormat": {
      "foregroundColor": {
        "red": 0.0,
        "green": 0.0,
        "blue": 0.0
      },
      "fontSize": 12,
      "bold": True,
    }
})

{'spreadsheetId': '15f4zO_W-LZCwkAnmCnpD9vHJOz6aOdQC0fdQ938frOU',
 'replies': [{}]}

It would be nice to add background gradient to spreadsheet similar to what could be done in pandas

In [ ]:
from matplotlib import pyplot

df.style.background_gradient(cmap=pyplot.get_cmap('Spectral')).format(precision=4)

,Country name,Ladder score,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,Finland,7.8042,10.7920,0.9688,71.1500,0.9614,-0.0188,0.1817
1,Denmark,7.5864,10.9622,0.9541,71.2501,0.9335,0.1342,0.1958
2,Iceland,7.5296,10.8955,0.9825,72.0500,0.9363,0.2110,0.6678
3,Israel,7.4729,10.6387,0.9433,72.6972,0.8089,-0.0231,0.7081
4,Netherlands,7.4030,10.9423,0.9305,71.5500,0.8869,0.2127,0.3789
5,Sweden,7.3952,10.8828,0.9392,72.1502,0.9480,0.1648,0.2024
6,Norway,7.3155,11.0877,0.9435,71.5000,0.9466,0.1406,0.2827
7,Switzerland,7.2401,11.1640,0.9204,72.9000,0.8911,0.0267,0.2663
8,Luxembourg,7.2279,11.6599,0.8785,71.6750,0.9149,0.0239,0.3450
9,New Zealand,7.1229,10.6619,0.9524,70.3501,0.8866,0.1748,0.2712


In [ ]:
import matplotlib
import numpy as np

# https://scipy-cookbook.readthedocs.io/items/Matplotlib_ColormapTransformations.html
def cmap_map(function, cmap):
    """ Applies function (which should operate on vectors of shape 3: [r, g, b]), on colormap cmap.
    This routine will break any discontinuous points in a colormap.
    """
    cdict = cmap._segmentdata
    step_dict = {}
    # Firt get the list of points where the segments start or end
    for key in ('red', 'green', 'blue'):
        step_dict[key] = list(map(lambda x: x[0], cdict[key]))
    step_list = sum(step_dict.values(), [])
    step_list = np.array(list(set(step_list)))
    # Then compute the LUT, and apply the function to the LUT
    reduced_cmap = lambda step : np.array(cmap(step)[0:3])
    old_LUT = np.array(list(map(reduced_cmap, step_list)))
    new_LUT = np.array(list(map(function, old_LUT)))
    # Now try to make a minimal segment definition of the new LUT
    cdict = {}
    for i, key in enumerate(['red','green','blue']):
        this_cdict = {}
        for j, step in enumerate(step_list):
            if step in step_dict[key]:
                this_cdict[step] = new_LUT[j, i]
            elif new_LUT[j,i] != old_LUT[j, i]:
                this_cdict[step] = new_LUT[j, i]
        colorvector = list(map(lambda x: x + (x[1], ), this_cdict.items()))
        colorvector.sort()
        cdict[key] = colorvector

    return matplotlib.colors.LinearSegmentedColormap('colormap',cdict,1024)

In [ ]:
from matplotlib import colors

spectral_cmap = pyplot.get_cmap('Spectral')
light_spectral_cmap = cmap_map(lambda x: x/2 + 0.5, spectral_cmap)

formats = []
for col_index in range(1, len(df.columns)): #len(df.columns)-1):
  col = df.iloc[:, col_index]
  min_value = col.min()
  max_value = col.max()
  norm = colors.Normalize(vmin=min_value, vmax=max_value, clip=True)

  idx = 2

  for value in col:
    normanized_value = norm(value)
    cell_id = f"{index_to_spreadsheet_column(col_index)}{idx}"
    color = light_spectral_cmap(normanized_value)
    formats.append({
        "range": f"{cell_id}:{cell_id}",
        "format": {
            "backgroundColor": {
              "red": color[0],
              "green": color[1],
              "blue": color[2]
            }
        }
    })
    idx += 1

  batch_format_response = sheet1.batch_format(formats)

In [ ]:
df.to_csv

<bound method NDFrame.to_csv of          Country name  Ladder score  Logged GDP per capita  Social support  \
0             Finland        7.8042              10.792010        0.968770   
1             Denmark        7.5864              10.962164        0.954112   
2             Iceland        7.5296              10.895531        0.982533   
3              Israel        7.4729              10.638705        0.943344   
4         Netherlands        7.4030              10.942279        0.930499   
..                ...           ...                    ...             ...   
132  Congo (Kinshasa)        3.2072               7.006671        0.651610   
133          Zimbabwe        3.2035               7.640998        0.689918   
134      Sierra Leone        3.1376               7.394014        0.555251   
135           Lebanon        2.3922               9.477677        0.529754   
136       Afghanistan        1.8590               7.324032        0.341341   

     Healthy life expectancy  F